In [1]:
import pandas as pd

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

import pickle

import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)

In [7]:
# pip install pandas scikit-learn xgboost pyspark numpy matplotlib imblearn

### Note:

- This is the first draft and work in progress, feel free to use the code for further use.

### Next steps
- Merge both train and test (done)
- Look into each of the columns (done)
- over_sampling/under_sampling for imbalance of class (is_fraud) (To-do)
- feature selection (to-do)
- split data into train and test (done)
- model building and evaluation (done)
- wrap the complete model into a dash app (to-do)


In [2]:
df_train = pd.read_csv('./data/fraudTrain.csv', low_memory=False, index_col=0)
df_test = pd.read_csv('./data/fraudTest.csv', low_memory=False, index_col=0)

In [3]:
df_train.shape, df_test.shape

((1296675, 22), (555719, 22))

In [4]:
list(df_train.columns) == list(df_test.columns)

True

In [5]:
df = pd.concat([df_train, df_test],ignore_index=True)

In [6]:
df.shape

(1852394, 22)

In [7]:
df.is_fraud.value_counts()

is_fraud
0    1842743
1       9651
Name: count, dtype: int64

### Looking into columns and cleaning

In [8]:
def clean_df(df):
    return df.drop(['cc_num','first', 'last', 'street', 'city', 'state', 'zip', 'dob', 'trans_num','trans_date_trans_time'],axis=1)

df = clean_df(df)

In [9]:
df.head(2)

,merchant,category,amt,gender,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,36.011293,-82.048315,0
1,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,1325376044,49.159047,-118.186462,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   merchant    object 
 1   category    object 
 2   amt         float64
 3   gender      object 
 4   lat         float64
 5   long        float64
 6   city_pop    int64  
 7   job         object 
 8   unix_time   int64  
 9   merch_lat   float64
 10  merch_long  float64
 11  is_fraud    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 169.6+ MB


### Sampling and handling the minority class using SMOTE

### Splitting data into train and test

In [11]:
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.shape, test.shape

((1481915, 12), (370479, 12))

In [12]:
train.to_csv('./data/clean_train.csv')
test.to_csv('./data/clean_test.csv')

### Encoding object columns

In [18]:
def encode(df):
    df_obj = df.select_dtypes(include=['object'])
    encoders = {}
    for col in df_obj.columns:
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])
        encoders[col] = encoder
    with open('LE_mdl_v1.pkl', 'wb') as f:
        pickle.dump(encoders, f)
    return df

train = encode(train)

In [19]:
train.head(2)

,merchant,category,amt,gender,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,29,4,166.80,1,39.3426,-114.8859,450,254,1371108903,40.088507,-113.895268,0
1,536,11,28.86,0,34.3795,-118.5230,34882,219,1347476946,35.356925,-119.348148,0


### Modelling

In [21]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

In [22]:
model1 = LogisticRegression()
model2 = RandomForestClassifier()
model3 = DecisionTreeClassifier()
model4 = xgb.XGBClassifier()

In [23]:
def model_train(model, x_train, y_train, x_test, y_test):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print('Accuracy Score: ',accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    with open(str(model)[:3] + '_mdl.pkl', 'wb') as f:
        pickle.dump(model,f)

In [24]:
model_train(model1, x_train, y_train, x_val, y_val)

Accuracy Score:  0.9941629580643964
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    294805
           1       0.00      0.00      0.00      1578

    accuracy                           0.99    296383
   macro avg       0.50      0.50      0.50    296383
weighted avg       0.99      0.99      0.99    296383



In [25]:
model_train(model2, x_train, y_train, x_val, y_val)

KeyboardInterrupt: 

In [26]:
model_train(model3, x_train, y_train, x_val, y_val)

KeyboardInterrupt: 

In [27]:
model_train(model4, x_train, y_train, x_val, y_val)

Accuracy Score:  0.9977427855173879
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    294805
           1       0.88      0.67      0.76      1578

    accuracy                           1.00    296383
   macro avg       0.94      0.83      0.88    296383
weighted avg       1.00      1.00      1.00    296383



In [31]:
df_train.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [33]:
df_train.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [34]:
df.columns

Index(['merchant', 'category', 'amt', 'gender', 'lat', 'long', 'city_pop',
       'job', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')